In [ ]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
from energyemissionsregio.config import SHP_PATH
import matplotlib.pyplot as plt
from cmcrameri import cm

In [ ]:
nuts0 = gpd.read_file(os.path.join(SHP_PATH, "NUTS0.shp"))
lau = gpd.read_file(os.path.join(SHP_PATH, "LAU.shp"))

In [ ]:
lau["country_code"] = lau["code"].str[:2]

In [ ]:
n_laus = lau[["country_code"]].copy()
n_laus["n_laus"] = 1

n_laus = n_laus.groupby("country_code").sum().reset_index()

In [ ]:
nuts0 = nuts0.merge(n_laus, left_on = "code", right_on='country_code', how='left')

In [ ]:
CoM_eu_countries = ['Belgium', 'Bulgaria',  'Cyprus', 'Italy', 'Czech Republic',
       'Germany', 'Denmark',  'Estonia', 'Spain', 'Finland', 'France',
       'Greece', 'Hungary','Ireland', 'Croatia', 'Lithuania', 'Luxembourg', 'Latvia', 
       'Romania', 'Malta', 'The Netherlands', 'Poland', 'Portugal', 
       'Sweden', 'Slovenia', 'Slovakia', 'Austria',]

In [ ]:
com_df = pd.read_csv(os.path.join("..", "data", "cities_2025-05-23.csv"))

In [ ]:
# com_eu_df = com_df[com_df['country'].isin(CoM_eu_countries)][['country', 'name', 'emissionsTotal', 'emissionsBuildings', 'emissionsTransportation', 'emissionsIndustry', 'emissionsWaste', 'emissionsOther',]]

com_eu_df = com_df[com_df['country'].isin(CoM_eu_countries)][['country']]

In [ ]:
com_eu_df["n_signatories"] = 1

In [ ]:
com_eu_n_signatories = com_eu_df.groupby("country").sum().reset_index()

In [ ]:
com_eu_n_signatories['country'] = com_eu_n_signatories['country'].replace({
    'Czech Republic': 'Czechia',
    'The Netherlands': 'Netherlands',
})

In [ ]:
nuts0 = nuts0.merge(com_eu_n_signatories, left_on = "name", right_on='country', how='left')

In [ ]:
nuts0.crs

In [ ]:
nuts0 = nuts0.to_crs("epsg:4326")

In [ ]:
nuts0['centroid'] = nuts0.geometry.centroid

# Extract x and y from centroid
nuts0['x'] = nuts0['centroid'].x
nuts0['y'] = nuts0['centroid'].y

# Plot base map
fig, ax = plt.subplots(figsize=(6, 5))
fig.patch.set_alpha(0)  # Transparent figure background
ax.set_facecolor('none')  # Transparent axes background
# nuts0.plot(ax=ax, color="white", edgecolor='black')

# Plot bubbles at centroids
scatter = ax.scatter(
    nuts0['x'], nuts0['y'],
    s=nuts0['n_laus'] * 0.2,                # scale size
    c=nuts0['n_signatories'], cmap=cm.batlow_r,    # color map
    alpha=0.6, edgecolors='black'
)

# Add colorbar
cbar = plt.colorbar(scatter, ax=ax)


# Final touches

ax.axis('off')

plt.savefig(os.path.join("..", "figures", "presentation", "com_signatories.png"),
            format='png', bbox_inches="tight", dpi=300, transparent=True)

plt.show()